In [1]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

2022-09-29 02:43:17.027241: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-29 02:43:17.127438: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-29 02:43:17.127460: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-09-29 02:43:17.150939: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-29 02:43:17.692198: W tensorflow/stream_executor/platform/de

In [2]:
# image = PIL.Image.open('train_gen/burn through/frame_00031.png')
batch_size = 32
# img_height = 974
# img_width = 800
img_height = 100
img_width = 100

In [3]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  'train_gen',
#     rescale=1.0/255.0
  validation_split=0.2,
  subset="training",
  seed=0,
  image_size=(img_height, img_width),
#     target_size = (100,100),
    shuffle = False,
  batch_size=batch_size)

Found 9718 files belonging to 6 classes.
Using 7775 files for training.


2022-09-29 02:43:26.260713: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-29 02:43:26.260880: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-29 02:43:26.260934: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-09-29 02:43:26.260985: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-09-29 02:43:26.261034: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

In [4]:
class_names = train_ds.class_names
print(class_names)

['burn through', 'contamination', 'good weld', 'lack of fusion', 'lack of penetration', 'misalignment']


In [5]:
# import matplotlib.pyplot as plt

# plt.figure(figsize=(10, 10))
# for images, labels in train_ds.take(1):
#       for i in range(9):
#         ax = plt.subplot(3, 3, i + 1)
#         plt.imshow(images[i].numpy().astype("uint8"))
#         plt.title(class_names[labels[i]])
#         plt.axis("off")

In [6]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(100).prefetch(buffer_size=AUTOTUNE)
# val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [7]:
normalization_layer = layers.Rescaling(1./255)

In [8]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
# image_batch, labels_batch = next(iter(normalized_ds))
# first_image = image_batch[0]
# # Notice the pixel values are now in `[0,1]`.
# print(np.min(first_image), np.max(first_image))

In [9]:
num_classes = len(class_names)

model = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [10]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 100, 100, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 100, 100, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 50, 50, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 50, 50, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 25, 25, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 25, 25, 64)        1

In [12]:
epochs=100
history = model.fit(
  normalized_ds,
#   validation_data=val_ds,
  epochs=epochs,verbose = 2,
    shuffle = False,
    use_multiprocessing = True 
)

Epoch 1/100
243/243 - 26s - loss: 1.5210 - accuracy: 0.3177 - 26s/epoch - 108ms/step
Epoch 2/100
243/243 - 16s - loss: 1.5782 - accuracy: 0.3336 - 16s/epoch - 64ms/step
Epoch 3/100
243/243 - 15s - loss: 1.5669 - accuracy: 0.2675 - 15s/epoch - 63ms/step
Epoch 4/100
243/243 - 15s - loss: 1.5584 - accuracy: 0.2835 - 15s/epoch - 63ms/step
Epoch 5/100
243/243 - 15s - loss: 1.5657 - accuracy: 0.2794 - 15s/epoch - 63ms/step
Epoch 6/100
243/243 - 16s - loss: 1.5574 - accuracy: 0.3164 - 16s/epoch - 64ms/step
Epoch 7/100
243/243 - 16s - loss: 1.5583 - accuracy: 0.3164 - 16s/epoch - 64ms/step
Epoch 8/100
243/243 - 16s - loss: 1.5543 - accuracy: 0.3164 - 16s/epoch - 66ms/step
Epoch 9/100
243/243 - 16s - loss: 1.5540 - accuracy: 0.3164 - 16s/epoch - 66ms/step
Epoch 10/100
243/243 - 16s - loss: 1.5612 - accuracy: 0.3164 - 16s/epoch - 65ms/step
Epoch 11/100
243/243 - 16s - loss: 1.5561 - accuracy: 0.3164 - 16s/epoch - 65ms/step
Epoch 12/100
243/243 - 18s - loss: 1.5635 - accuracy: 0.3164 - 18s/epoch 

Epoch 98/100
243/243 - 16s - loss: 1.5445 - accuracy: 0.3164 - 16s/epoch - 67ms/step
Epoch 99/100
243/243 - 16s - loss: 1.5440 - accuracy: 0.3164 - 16s/epoch - 66ms/step
Epoch 100/100
243/243 - 16s - loss: 1.5446 - accuracy: 0.3164 - 16s/epoch - 67ms/step


In [13]:
model.save('Al_defect_model')

INFO:tensorflow:Assets written to: Al_defect_model/assets


INFO:tensorflow:Assets written to: Al_defect_model/assets
